---
title: "RH Analytics"
sutitle: "Preparando o Conjunto de Dados de Treinamento"
author: "Sérgio Carvalho"
date: "02 setembro, 2019"
output:
  rmdformats::readthedown:
    self_contained: true
    thumbnails: true
    lightbox: true
    gallery: false
    highlight: zenburn 
    code_folding: show
    style_body: justify
    df_print: paged
    number_sections: yes
    toc_float:
      collapsed: yes
      smooth_scroll: yes
editor_options: 
    chunk_output_type: inline
---









# Objetivos

  * Número de variáveis: 18 
  * Tipo de variáveis
      * * Inteiras ou discretas: 
      * * Numéricas ou double
      * * Categóricas
      * * Qualitativas
   * Qualidade dos dados          
      * * Quantidade de NA's por variável
  * Criação de novas variáveis, se precisar
  * Transformação das variáveis, se precisar



# Lendo os Conjuntos de Dados

In [ ]:
%%R
reticulate::source_python("../pickle_reader.py")
df.train <- read_pickle_file("../dataset/base_treinamento_testes.pkl") %>%
                                          data.frame(stringsAsFactors = F)

df.sub <- read_pickle_file("../dataset/base_submission.pkl") %>%
                                            data.frame(stringsAsFactors = F)

## Base de treinamento

In [ ]:
%%R
df.train %>% head(10)

## Base de Submissão

In [ ]:
%%R
df.sub %>% head(10)

Adicionando a variável **aprovado_vaga** na base de submissão.

In [ ]:
%%R
df.sub$aprovado_vaga <- NA
df.sub %>% head(20)

## Unindo as bases 

Agora vou juntar as bases de treinamento e submissão para tratar os dados de forma uniforme.

In [ ]:
%%R
df.all <- rbind(df.train,df.sub)

## Há quantos tipo de dados ?

In [ ]:
%%R
lapply(df.all,class) %>%
                unlist %>% 
          as.character %>% 
                 table

Quais são essas colunas com o tipo de dados lista?

## Colunas com tipo de dados __list__.

In [ ]:
%%R
df.all[,lapply(df.all,class) == 'list'] %>% head() 

### Recuperando as informações das colunas com tipo de dados __list__.

In [ ]:
%%R
df.all[,lapply(df.all,class) == 'list'] <- df.all[,lapply(df.all,class) == 'list'] %>%
                                                                    lapply('as.character') %>%
                                                                    data.frame()      

E agora como estão os metadados?

In [ ]:
%%R
lapply(df.all,class) %>%
                unlist %>% 
          as.character %>% 
                 table

## Analisando as colunas pelo tipo de dados.

### Tipo de dado String

In [ ]:
%%R
df.all[,lapply(df.all,class) == 'character'] %>% head()

Vamos transformar o tipo de dados dessas variáveis de string para categóricas.

In [ ]:
%%R
df.all[,lapply(df.all,class) == 'character'] = df.all[,lapply(df.all,class) == 'character'] %>% 
                                                                                    apply(2,factor) %>% 
                                                                   data.frame(stringsAsFactors = T)

Aproveito para converter o tipo de dado da variável aprovado_vaga, tipo inteiro, para categórico.

In [ ]:
%%R
df.all$aprovado_vaga <- as.factor(df.all$aprovado_vaga)

E agora, quais são os tipo de dados no data-frame ?

In [ ]:
%%R
lapply(df.all,class) %>%
                unlist %>% 
          as.character %>% 
                 table

overview das caracteríscas dos dados.

In [ ]:
%%R
str(df.all)

### Tipo de dado Categórico

In [ ]:
%%R
df.fator <- df.all[,lapply(df.all,class) == 'factor'] 

informações resumida dos dados.

In [ ]:
%%R
df.fator %>% 
      summary

* Observe que nas variáveis __area_interesse_candidato__, __cargo_vaga e cidade_vaga__ há um números significativo de valores nulos, esses valores podem ser tratados por uma série de técnicas que vão desde o imput através de modelagem até a sua exclusão.  

  * Outro ponto, é o fato de a variável __nível_vaga__ ter apenas 1 categoria __'JUNIOR/TRAINEE'__, logo iremos excluí-la de nossa análise, pois com apenas uma categoria ou valor qualquer variável não nos fornece informações relevantes. 

  * As categorias da variável de interesse estão desbalanceadas, observe que o número de eventos para a categoria 1 é um pouco maior que 10% do número de eventos da categoria 0, isto implica que pode ser necessário algum processo de amostragem para atingir uma performance razoável como output do processo de modelagem. 



### Tipo de Dado Numérico

In [ ]:
%%R
df.num <- df.all[,lapply(df.all,class) == 'numeric'] 
df.num %>% head()

### Resumo dos dados numéricos

In [ ]:
%%R
summary(df.num)

De forma geral os valores dessas variáveis apresentam um comportamento bem distribuído em torno de sua média/mediana e isso ajuda bastante no processo de convergência para a(s) estimativa(s) dos valore(s) dos parâmetro(s) que cada método de aprendizagem dispoe. No entanto, as variáveis __ultimo_salario_candidato__ e __mediana_teste_espanhol_candidato__ apresentam pontos que estão de certa forma distantes de suas médias/mediana podendo causar assimetria em seus comportamentos e como consequência uma certa dificuldade em se realizar predições sobre eles. 

Os pontos extremos, outliers, muitas vezes são retirados dos conjuntos de dados de forma equivocada e quando isso ocorre o modelo passa a não representar a realidade do evento de interesse limitando-se a predições compreendidas dentro de um range menor de valores o que certamente prejudica a tomada de decisão.


## A variável __codigo_vaga__

Será que podemos transformar os códigos desssas vagas em categorias? Afinal, há quantos códigos de vagas?

In [ ]:
%%R
table(df.num$codigo_vaga)

veja que interessante há 27 códigos diferentes, iremos substituí-los por letras e converter esta variável em categórica.

In [ ]:
%%R
set.seed(497)
df.num$codigo_vaga <- as.character(df.num$codigo_vaga)
(codigos <- data.frame(codigos = df.num$codigo_vaga %>% 
                                              table %>% 
                                               names,
                       letras = as.character(c(LETTERS[1:26],
                                               paste(LETTERS[1:2],
                                                     collapse = ""))),
                                stringsAsFactors = F))

substituindo os códigos da variável __codigo_vaga__ por letras.

In [ ]:
%%R
for(i in 1:nrow(codigos)) df.num[df.num$codigo_vaga == codigos[i,1],'codigo_vaga'] <- as.character(codigos[i,2]) 

In [ ]:
%%R
df.num %>% head

## Dados Categóricos + Numéricos

In [ ]:
%%R
df.all = bind_cols(df.fator,df.num) %>% 
         select(aprovado_vaga,codigo_vaga, everything()) %>%
            data.frame(stringsAsFactors = T)

In [ ]:
%%R
xtabs( ~ aprovado_vaga + codigo_vaga, 
           data = df.all[!is.na(df.all$aprovado_vaga),]) %>% 
           prop.table(2) %>% 
           barplot(legend = T, horiz = F, las = 1, main = '\n Tabela das Proporções Condicionais das Variáveis codigo_vaga vs aprovado_vaga \n')

Observe no gráfico acima que dependendo do código da vaga em que o candidado se inscrever ele terá maior ou menor chance de ser aprovado, esta variável poderá nos ajudar a discriminar a variável resposta __aprovado_vaga__.   

## Resumo dos dados

In [ ]:
%%R
skim_to_wide(df.all)

Como os valores nulos foram transformados em categóricos a coluna missing nos diz que não há dados nulos em nosso dataframe.

In [ ]:
%%R
df.all$codigo_vaga <- as.factor(df.all$codigo_vaga)

Qual a importância de cada variável em relação a variável resposta.
 
## Importância das Variáveis

In [ ]:
%%R
importance.vars(df.all,'aprovado_vaga')

Aqui temos uma idéia prévia das variáveis que poderão fazer parte do modelo, veja que as variáveis __ultimo_cargo_candidato__, __mediana_teste_ingles_candidato__, __cidade_candidato__ podem contribuir significativamente para nos ajudar a explicar o comportamento da variável resposta.  


## Número de Categoria das Variáveis

In [ ]:
%%R
apply(df.all,2,function(x) length(table(x))) %>% sort(decreasing = T) 

Note que as variáveis __ultimo_cargo_candidato__, __cidade_candidato__, __area_interesse_candidato__ possuem muitas categorias e isto pode ser um problema quando formos realizar a modelagem, vamos analisá-las. 

## Variável __ultimo_cargo_candidato__

Quais são as categorias da variável __ultimo_cargo_candidato__?

In [ ]:
%%R
df.all$ultimo_cargo_candidato %>% 
                        table %>% 
         sort(decreasing = T) %>%
                     head(50)

Com uma rápida olhada nestas categorias é possível observar que as palavras __ESTAGIO__, __ENGENHEIRO__, __ANALISTA__, __TRAINEE__ entre outras são bastante frequentes, precisamos trata-las afim de reduzir o número de categorias desta variável.       
Agora vamos fazer um split em cada string da variável __ultimo_cargo_candidato__ e por fim contar verficar as categorias mais frequentes.

In [ ]:
%%R
df.all$ultimo_cargo_candidato %>% 
                 str_split(" ") %>% 
                         unlist %>%
                        table() %>% 
           sort(decreasing = T) %>% 
                       head(50)

criando uma variável.

In [ ]:
%%R
df.all$emprego <- NA 
df.all$emprego <- as.character(df.all$emprego)

In [ ]:
%%R
(df.cargo <- readxl::read_excel('../dataset/ultimo_cargo.xlsx',sheet = 'Plan1'))  

imputando as novas categorias referente ao último cargo do candidato.

In [ ]:
%%R
for(i in 1:nrow(df.cargo)){ 

  df.all[str_detect(as.character(df.all[,3]),as.character(df.cargo[i,1])),'emprego'] <- as.character(df.cargo[i,2])
}

In [ ]:
%%R
df.all %>% 
  select(ultimo_cargo_candidato,emprego)

E agora ficamos com quantas categorias?

In [ ]:
%%R
df.all$emprego %>% 
  table %>% 
  sort(decreasing = T)

agora temos apenas 25 categorias, ocorre porém que este processo de redução das categorias implica em termos uma menor variabilidade da variável de interesse em relação a essas novas categoricas, o que de certa forma acaba prejudicando o desempenho dos modelos, o ideal é encontrar o número mínimo de categorias que maximizam a variabilidade da variável resposta.

In [ ]:
%%R
sum(df.all$emprego %>% is.na())/nrow(df.all)

a categoria **OUTROS** irá representar as categorias menos frequentes.

In [ ]:
%%R
df.all$emprego[is.na(df.all$emprego)] <- 'OUTROS'

atribuindo as novas categorias a variável **ultimo_cargo_candidato**.

In [ ]:
%%R
df.all$ultimo_cargo_candidato <- df.all$emprego

deletando a variável emprego.

In [ ]:
%%R
df.all$emprego <- NULL

vejamos como ficou nosso dataframe.

In [ ]:
%%R
df.all %>% head()

## Variável __area_interesse_candidato__


Com uma rápida olhada nestas categorias é possível observar que as palavras __ENGENHARIA__, __ADMINISTRACAO__, __PRODUCAO__, __COMERCIAL/VENDAS__ entre outras são bastante frequentes, precisamos tratá-las afim de reduzir o número de categorias destas variáveis.     

Agora vamos fazer um split em cada string da variável __area_interesse_candidato__ e por fim contar verficar as categorias mais frequentes.

In [ ]:
%%R
area <- df.all$area_interesse_candidato %>% 
                         str_split(" ") %>% 
                                 unlist %>%
                                table() %>% 
                   sort(decreasing = T) %>% 
                                names()

a função relationship recebe uma palavra e uma vetor de strings e nos retorna o contexto da palavra para melhor categoriza-la.

In [ ]:
%%R
y <- as.character(df.all$area_interesse_candidato)
relationship(area[1],y)

In [ ]:
%%R
df.area <- readxl::read_excel('../dataset/area_interesse.xlsx',sheet = 'Plan1')  

criando uma variável denomida de __area__.

In [ ]:
%%R
df.all$area <- NA 

imputando as novas categorias referente a área de interesse do candidato.

In [ ]:
%%R
for(i in 1:nrow(df.area)){ 
  
  df.all[str_detect(as.character(df.all[,4]),as.character(df.area[i,1])),'area'] <- as.character(df.area[i,2])
}

In [ ]:
%%R
df.all %>% 
  select(cidade_candidato,area)

porcentagem de valores nulos **NA** em **area**.

In [ ]:
%%R
sum(df.all$area %>% is.na())/nrow(df.all)

as categorias de menor frequência serão representadas pela categoria  **OUTRAS**.

In [ ]:
%%R
df.all$area[is.na(df.all$area)] <- 'OUTROS'

atribuindo as novas categorias a variável **area_interesse_candidato**.

In [ ]:
%%R
df.all$area_interesse_candidato <- df.all$area

deletando a variável area

In [ ]:
%%R
df.all$area <- NULL

Selecinando algumas variáveis.

In [ ]:
%%R
df.all <- df.all %>% 
           select(-nivel_vaga, 
                  -formacao_vaga,
                  -cidade_candidato)

dados de treinamento e submissão.

In [ ]:
%%R
df.train <- df.all[!is.na(df.all$aprovado_vaga),]
df.sub <- df.all[is.na(df.all$aprovado_vaga),]

# Exportando os dados

In [ ]:
%%R
write.csv(df.train,'../outputs/df.train.csv')
write.csv(df.sub,'../outputs/df.sub.csv')